First, create a new conda environment named BI2025 and install the required packages from requirements.txt


In [1]:
#!conda create -n BI2025 python=3.11 -y
#!conda activate BI2025
#!pip install -r requirements.txt

In [2]:
# DO NOT MODIFY OR COPY THIS CELL!! 
# Note: The only imports allowed are Python's standard library, pandas, numpy, scipy, matplotlib, seaborn and scikit-learn
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
import typing
import requests
import time
import shutil
import json
from starvers.starvers import TripleStoreEngine

## Graph-based documentation preliminaries

**!!!IMPORTANT!!!**

Everytime you work on this notebook, enter your student ID in the `executed_by` variable so that the cell executions are accredited to you.

In [3]:
executed_by ='stud-id_52253331'  # Replace the digits after "id_" with your own student ID

Set your group and student IDs. Do this only once.

In [4]:
# group id for this project
group_id = '05'  # Replace the digits with your group id

# Students working on this notebook
student_a = 'stud-id_52253331'  # Replace the digits after "id_" with student A's student ID
student_b = 'stud-id_12426894'  # Replace the digits after "id_" with student B's student ID

In [5]:
# Roles. Don't change these values.
code_writer_role = 'code_writer'
code_executor_role = 'code_executor'

Setup the starvers API for logging your steps into our server-sided graph database.

In [6]:
get_endpoint = "https://starvers.ec.tuwien.ac.at/BI2025"
post_endpoint = "https://starvers.ec.tuwien.ac.at/BI2025/statements"
engine = TripleStoreEngine(get_endpoint, post_endpoint, skip_connection_test=True)

Use these prefixes in your notebooks. You can extend this dict with your prefixes of additional ontologies that you use in this notebook. Replace 00 with your group id

In [7]:
prefixes = {
    'xsd': 'http://www.w3.org/2001/XMLSchema#',
    'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
    'foaf': 'http://xmlns.com/foaf/0.1/',
    'prov': 'http://www.w3.org/ns/prov#',
    'sc': 'https://schema.org/',
    'cr': 'http://mlcommons.org/croissant/',
    'mls': 'http://www.w3.org/ns/mls#',
    'mlso': 'http://w3id.org/mlso',
    'siu': 'https://si-digital-framework.org/SI/units/',
    'siq': 'https://si-digital-framework.org/SI/quantities/',
    'qudt': 'http://qudt.org/schema/qudt/',
    '': f'https://starvers.ec.tuwien.ac.at/BI2025/{group_id}/',
}

prefix_header = '\n'.join([f'PREFIX {k}: <{v}>' for k, v in prefixes.items()]) + '\n\n'

Ontologies to use
* Provenance of the experiment process
    * PROV-O: 
        * doc: https://www.w3.org/TR/prov-o/
        * serialization: https://www.w3.org/ns/prov-o
* Data used and created
    * schema.org - Dataset: 
        * doc: https://schema.org/Dataset
        * serialization: https://schema.org/version/latest/schemaorg-current-https.ttl
    * Crossaint
        * doc: https://docs.mlcommons.org/croissant/docs/croissant-spec.html
        * serialization: https://github.com/mlcommons/croissant/blob/main/docs/croissant.ttl
* ML experiments performed
    * MLSO: 
        * doc: https://github.com/dtai-kg/MLSO
        * doc: https://dtai-kg.github.io/MLSO/#http://w3id.org/
        * serialization: https://dtai-kg.github.io/MLSO/ontology.ttl
* Measurements, Metrics, Units
    * QUDT
        * doc:https://qudt.org/
        * doc: https://github.com/qudt/qudt-public-repo
        * serialization: https://github.com/qudt/qudt-public-repo/blob/main/src/main/rdf/schema/SCHEMA_QUDT.ttl
    * SI Digital Framework
        * doc: https://github.com/TheBIPM/SI_Digital_Framework/blob/main/SI_Reference_Point/docs/README.md
        * doc: https://si-digital-framework.org/
        * doc: https://si-digital-framework.org/SI
        * serialization: https://github.com/TheBIPM/SI_Digital_Framework/blob/main/SI_Reference_Point/TTL/si.ttl
    * Quantities and Units
        * doc: https://www.omg.org/spec/Commons
        * serialization: https://www.omg.org/spec/Commons/QuantitiesAndUnits.ttl

Use this function to record execution times.

In [8]:
def now() -> str:
    """
    Returns the current time in ISO 8601 format with UTC timezone in the following format:
    YYYY-MM-DDTHH:MM:SS.sssZ
    """
    timestamp = datetime.datetime.now(datetime.timezone.utc)
    timestamp_formated = timestamp.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]  +"Z"

    return timestamp_formated

Register yourself in the Knowledge Graph using ProvO. Change the given name, family name and immatriculation number to reflect your own data.

In [9]:
# Ontologies used: foaf, prov, IAO
reigstration_triples_a = [
f':{student_a} rdf:type foaf:Person .',
f':{student_a} rdf:type prov:Agent .',
f':{student_a} foaf:givenName "Ana" .',
f':{student_a} foaf:familyName "Zrnic" .',
f':{student_a} <http://vivoweb.org/ontology/core#identifier> "{student_a}" .',
f':{student_a} rdf:type <http://purl.obolibrary.org/obo/IAO_0000578> .',
f':{student_a} <http://www.w3.org/2000/01/rdf-schema#label> "Immatriculation number" .',
f':{student_a} <http://purl.obolibrary.org/obo/IAO_0000219> "52253331"^^xsd:string .',
]

reigstration_triples_b = [
f':{student_b} rdf:type foaf:Person .',
f':{student_b} rdf:type prov:Agent .',
f':{student_b} foaf:givenName "Daria" .',
f':{student_b} foaf:familyName "Alekseienkova" .',
f':{student_b} <http://vivoweb.org/ontology/core#identifier> "{student_b}" .',
f':{student_b} rdf:type <http://purl.obolibrary.org/obo/IAO_0000578> .',
f':{student_b} <http://www.w3.org/2000/01/rdf-schema#label> "Immatriculation number" .',
f':{student_b} <http://purl.obolibrary.org/obo/IAO_0000219> "12426894"^^xsd:string .',
]

role_triples = [
    f':{code_writer_role} rdf:type prov:Role .',
    f':{code_executor_role} rdf:type prov:Role .',
]


engine.insert(reigstration_triples_a, prefixes=prefixes)
engine.insert(reigstration_triples_b, prefixes=prefixes)
engine.insert(role_triples, prefixes=prefixes)

**What not do do**

Do not use [blank nodes](https://www.w3.org/wiki/BlankNodes).

PROV-O uses blank nodes to connect multiple elements with each other.
Such blank nodes (such as _:association) should not be used.
Instead, assign a fixed node ID such as
:5119fcd7-b571-41e0-9464-a37c7be0f574 by generating them outside of the
notebook.
We suggest that, for each setting where such a blank node is needed to
connect multiple elements, you create a unique hash (using uuid.uuid4())
and keep this as hard-coded identifier for the blank node. The template
notebook contains examples of this. Do *not* use these provided values,
as otherwise, your provenance documentations will all be connected via
these identifiers!
Also, do not generate them dynamically in every cell execution, e.g. by
using uuid.uuid4() in a cell. This would generate many new linking nodes
for connecting the same elements.
Compute one for each node (cell) where you need them and make sure to
use the same one on each re-execution of the notebook.

In [10]:
#weather_data_path = os.path.join("data", "datasets", "weather")
marketing_data_path = os.path.join("data", "datasets", "ifood")

def load_marketing_data():
    input_file = os.path.join(marketing_data_path, "ml_project1_data.csv")   # change name if needed
    df = pd.read_csv(input_file)
    # Example: print first rows
    display(df.head())
    return df
start_time_ld = now()
ifood_df = load_marketing_data()
end_time_ld = now()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0


## Business Understanding 

In [11]:
## Each Activity that follows is part of the Business Understanding Phase

business_understanding_phase_executor = [
f':business_understanding_phase rdf:type prov:Activity .',
f':business_understanding_phase rdfs:label "Business Understanding Phase" .', ## Phase 1: Business Understanding
]
engine.insert(business_understanding_phase_executor, prefixes=prefixes)


In [12]:
#############################################
# Documentation
#############################################

data_src_and_scenario_comment = """
This dataset was taken form the website Kaggle. It is a dataset that contains transactional and demographic information about customers of Ifood, a Brazilian online ordering and delivery platform.
The dataset originally comes from a public github repository when the Ifood Brain team had a data challenge for the data analyst role hiring process.
The company wants to understand the spending behavior of its customers to improve effectivness of marketing campaigns with historical customer interactions and their past spending patterns.
In order to achieve this, we are building a data-driven approach using data mining techniques to analyze customer data and identify patterns and trends in their spending behavior.
"""

business_objectives_comment = """
The primary business objective of the customer is to predict how much a customer is likely to spend, in order to imporve marketing strategies.
This allows for better targeting, bugdet allocation and campaing planning.
Key business questions: 
- Which variables (behavioral or demographic) are more significant in predicting customer spending?
- How much is a customer expected to spend based on their profile and past interactions?
"""

business_success_criteria_comment = """
Business success is achieved if the implemented prediction model improves the marketing team's effectiveness in targeting customers and allocating budgets.
Measurable criteria:
-Identify the top 20%% of customers who are the highest spenders before launching a new campaign.
-Increase revenue by 6%% in the next quarter compared to the previous quarter by targeting high-spending customers.
-Reduce marketing costs by at least 13%% on customers who are inactive.

Subjective criteria:
- The model's insights and predictions are positively received and understandable by the marketing team and marketing managers.
- The model is stable and doesn't drastically change within smaller time frames, which the stakeholders would find reassuring.
"""

data_mining_goals_comment = """
To support the business objectives, a regression model will be developed to predict customer spending, and to identify which features most strongly influence that spending.
The outputs of the data mining process will include:
- Predicted spending value per customer
- A ranked list of significant features influencing customer spending

What is not part of our data mining goals:
- Not predicting campaign success probability
"""

data_mining_success_criteria_comment = """
A successful outcome requires that the regression model achieves acceptable predictive performance, as measured by the following criteria:
- The coefficient of determination (R^2) is at least 0.65, indicating that a significant amount of the variance is explained by the input features.
- The Mean Absolute Error (MAE) value is below 15%% of the average spending value.
- The Root Mean Squared Error (RMSE) value is below 10%% of the average spending value.
"""

ai_risk_aspects_comment = """
According to the EU AI Regulation risk classification, this project falls under the 'minimal or no risk' category.
The dataset contains no sensitive personal or biometric data, reducing the potential for discrimination or harm, it is used exclusively for analytical purposes to improve marketing strategies.
No special regulatory requirements are applicable, beyond standard data protection and transparency practices to avoid bias.
"""


bu_ass_uuid_executor = "bb6a40f9-9d92-4f9f-bbd2-b65ef6a82da2" # Generate once
business_understanding_executor = [
f':business_understanding rdf:type prov:Activity .',
f':business_understanding sc:isPartOf :business_understanding_phase .', # Connect Activity to Parent Business Understanding Phase Activity
f':business_understanding prov:qualifiedAssociation :{bu_ass_uuid_executor} .',
f':{bu_ass_uuid_executor} prov:agent :{executed_by} .',
f':{bu_ass_uuid_executor} rdf:type prov:Association .',
f':{bu_ass_uuid_executor} prov:hadRole :{code_executor_role} .',
]
engine.insert(business_understanding_executor, prefixes=prefixes)


business_understanding_data_executor = [
# 1a
f':bu_data_source_and_scenario rdf:type prov:Entity .',
f':bu_data_source_and_scenario prov:wasGeneratedBy :business_understanding .',
f':bu_data_source_and_scenario rdfs:label "1a Data Source and Scenario" .',
f':bu_data_source_and_scenario rdfs:comment """{data_src_and_scenario_comment}""" .',
# 1b
f':bu_business_objectives rdf:type prov:Entity .',
f':bu_business_objectives prov:wasGeneratedBy :business_understanding .',
f':bu_business_objectives rdfs:label "1b Business Objectives" .',
f':bu_business_objectives rdfs:comment """{business_objectives_comment}""" .',
# 1c
f':bu_business_success_criteria rdf:type prov:Entity .',
f':bu_business_success_criteria prov:wasGeneratedBy :business_understanding .',
f':bu_business_success_criteria rdfs:label "1c Business Success Criteria" .',
f':bu_business_success_criteria rdfs:comment """{business_success_criteria_comment}""" .',
# 1d
f':bu_data_mining_goals rdf:type prov:Entity .',
f':bu_data_mining_goals prov:wasGeneratedBy :business_understanding .',
f':bu_data_mining_goals rdfs:label "1d Data Mining Goals" .',
f':bu_data_mining_goals rdfs:comment """{data_mining_goals_comment}""" .',
# 1e
f':bu_data_mining_success_criteria rdf:type prov:Entity .',
f':bu_data_mining_success_criteria prov:wasGeneratedBy :business_understanding .',
f':bu_data_mining_success_criteria rdfs:label "1e Data Mining Success Criteria" .',
f':bu_data_mining_success_criteria rdfs:comment """{data_mining_success_criteria_comment}""" .',
# 1f
f':bu_ai_risk_aspects rdf:type prov:Entity .',
f':bu_ai_risk_aspects prov:wasGeneratedBy :business_understanding .',
f':bu_ai_risk_aspects rdfs:label "1f AI risk aspects" .',
f':bu_ai_risk_aspects rdfs:comment """{ai_risk_aspects_comment}""" .',

]
engine.insert(business_understanding_data_executor, prefixes=prefixes)

## Data Understanding

The following pseudo-code & pseudo-documentation may be used as a hint.

In [13]:
## Each Activity that follows is part of the Data Understanding Phase

data_understanding_phase_executor = [
f':data_understanding_phase rdf:type prov:Activity .',
f':data_understanding_phase rdfs:label "Data Understanding Phase" .', 
]
engine.insert(data_understanding_phase_executor, prefixes=prefixes)


In [14]:
attribute_types = ifood_df.dtypes.astype(str)
raw_mapping = """
ID                       nominal
Year_Birth               interval
Education                ordinal
Marital_Status           nominal
Income                   ratio
Kidhome                  ratio
Teenhome                 ratio
Dt_Customer              interval
Recency                  ratio
MntWines                 ratio
MntFruits                ratio
MntMeatProducts          ratio
MntFishProducts          ratio
MntSweetProducts         ratio
MntGoldProds             ratio
NumDealsPurchases        ratio
NumWebPurchases          ratio
NumCatalogPurchases      ratio
NumStorePurchases        ratio
NumWebVisitsMonth        ratio
AcceptedCmp3             nominal-binary
AcceptedCmp4             nominal-binary
AcceptedCmp5             nominal-binary
AcceptedCmp1             nominal-binary
AcceptedCmp2             nominal-binary
Complain                 nominal-binary
Z_CostContact            unknown
Z_Revenue                unknown
Response                 nominal-binary
"""

mapping_dict = {}
for line in raw_mapping.strip().split("\n"):
    parts = line.split()
    attribute = parts[0]
    measurement = parts[-1]
    mapping_dict[attribute] = measurement

mapping_df = pd.DataFrame.from_dict(mapping_dict, orient='index', columns=['Measurement_Level'])
mapping_df = mapping_df.reset_index().rename(columns={'index': 'Attribute'})

# Create a combined view with data types and measurement levels
data_types_df = pd.DataFrame({
    'Attribute': ifood_df.columns,
    'Data_Type': ifood_df.dtypes.astype(str).values
})
data_types_df = data_types_df.merge(mapping_df, on='Attribute', how='left')
display(data_types_df)


#############################################
# Documentation: Load Data Activity
#############################################

ld_ass_uuid_executor = "a3c7e8f2-5b4d-4a9e-8c1f-3d2e6b7a9c0d"
load_data_code_writer = student_a

load_data_activity = [
    f':load_data rdf:type prov:Activity .',
    f':load_data sc:isPartOf :data_understanding_phase .',
    f':load_data rdfs:label "Load Marketing Data" .',
    f':load_data prov:startedAtTime "{start_time_ld}"^^xsd:dateTime .',
    f':load_data prov:endedAtTime "{end_time_ld}"^^xsd:dateTime .',
    f':load_data prov:qualifiedAssociation :{ld_ass_uuid_executor} .',
    f':{ld_ass_uuid_executor} prov:agent :{executed_by} .',
    f':{ld_ass_uuid_executor} rdf:type prov:Association .',
    f':{ld_ass_uuid_executor} prov:hadRole :{code_executor_role} .',
]
engine.insert(load_data_activity, prefixes=prefixes)

#############################################
# Documentation: Attribute Types Activity
#############################################

start_time_at = now()
# Attribute type analysis already performed above
end_time_at = now()

attribute_types_code_writer = student_a
at_ass_uuid_executor = "d4f8e2a1-7c3b-4e9d-8f5a-1b2c3d4e5f6a"  # Generate once
at_ass_uuid_writer = "e5a9f3b2-8d4c-5f0e-9a6b-2c3d4e5f6a7b"  # Generate once

at_comment = """
Analysis of attribute types in the iFood marketing dataset. Each attribute was classified as: nominal, ordinal, interval or ratio based on the nature of the data:
- Nominal: Categorical variables without inherent order (e.g., ID, Marital_Status)
- Ordinal: Categorical variables with meaningful order (e.g., Education)
- Interval: Numeric variables with meaningful differences but no true zero (e.g., Year_Birth, Dt_Customer)
- Ratio: Numeric variables with true zero point (e.g., Income, spending amounts, counts)
- Nominal-binary: Binary categorical variables (e.g., campaign acceptance flags, Complain, Response)
"""

# Document the attribute types activity
attribute_types_activity = [
    f':identify_attribute_types rdf:type prov:Activity .',
    f':identify_attribute_types sc:isPartOf :data_understanding_phase .',
    f':identify_attribute_types rdfs:label "Identify Attribute Types" .',
    f':identify_attribute_types rdfs:comment """{at_comment}""" .',
    f':identify_attribute_types prov:startedAtTime "{start_time_at}"^^xsd:dateTime .',
    f':identify_attribute_types prov:endedAtTime "{end_time_at}"^^xsd:dateTime .',
    
    # Executor association
    f':identify_attribute_types prov:qualifiedAssociation :{at_ass_uuid_executor} .',
    f':{at_ass_uuid_executor} prov:agent :{executed_by} .',
    f':{at_ass_uuid_executor} rdf:type prov:Association .',
    f':{at_ass_uuid_executor} prov:hadRole :{code_executor_role} .',
    
    # Writer association
    f':identify_attribute_types prov:qualifiedAssociation :{at_ass_uuid_writer} .',
    f':{at_ass_uuid_writer} prov:agent :{attribute_types_code_writer} .',
    f':{at_ass_uuid_writer} rdf:type prov:Association .',
    f':{at_ass_uuid_writer} prov:hadRole :{code_writer_role} .',
    
    # Input: raw data
    f':identify_attribute_types prov:used :raw_data .',
    
    # Output: attribute types report entity
    f':attribute_types_report rdf:type prov:Entity .',
    f':attribute_types_report prov:wasGeneratedBy :identify_attribute_types .',
    f':attribute_types_report rdfs:label "Attribute Types Classification Report" .',
    f':attribute_types_report rdfs:comment "Classification of 29 attributes into measurement scales: nominal, ordinal, interval, ratio, and nominal-binary." .',
]
engine.insert(attribute_types_activity, prefixes=prefixes)

# Document measurement level statistics
measurement_counts = mapping_df['Measurement_Level'].value_counts().to_dict()
measurement_stats_comment = f"""
Measurement level distribution:
- Ratio: {measurement_counts.get('ratio', 0)} attributes (numeric with true zero)
- Nominal-binary: {measurement_counts.get('nominal-binary', 0)} attributes (binary flags)
- Nominal: {measurement_counts.get('nominal', 0)} attributes (categorical without order)
- Interval: {measurement_counts.get('interval', 0)} attributes (numeric without true zero)
- Ordinal: {measurement_counts.get('ordinal', 0)} attributes (categorical with order)
- Unknown: {measurement_counts.get('unknown', 0)} attributes (to be investigated)
"""

attribute_types_stats = [
    f':attribute_types_statistics rdf:type prov:Entity .',
    f':attribute_types_statistics prov:wasGeneratedBy :identify_attribute_types .',
    f':attribute_types_statistics prov:wasDerivedFrom :attribute_types_report .',
    f':attribute_types_statistics rdfs:label "Attribute Types Statistics" .',
    f':attribute_types_statistics rdfs:comment """{measurement_stats_comment}""" .',
]
engine.insert(attribute_types_stats, prefixes=prefixes)

print("Attribute types documentation inserted successfully.")


,Attribute,Data_Type,Measurement_Level
0,ID,int64,nominal
1,Year_Birth,int64,interval
2,Education,object,ordinal
3,Marital_Status,object,nominal
4,Income,float64,ratio
5,Kidhome,int64,ratio
6,Teenhome,int64,ratio
7,Dt_Customer,object,interval
8,Recency,int64,ratio
9,MntWines,int64,ratio


Attribute types documentation inserted successfully.


In [15]:
def infer_unit(col, series):
    col_lower = col.lower()

    if series.dropna().nunique() == 2 and set(series.dropna().unique()).issubset({0,1}):
        return "binary indicator (0/1)"

    if "date" in col_lower or "dt" in col_lower:
        return "calendar date (YYYY-MM-DD)"

    if "recency" in col_lower:
        return "days"

    if any(x in col_lower for x in ["mnt", "meat", "fish", "wine", "fruits", "gold", "sweet"]):
        return "monetary amount (EUR)"

    if col_lower.startswith("num") or "visits" in col_lower or "purchases" in col_lower:
        return "count"

    if "kid" in col_lower or "teen" in col_lower or col_lower.endswith("home"):
        return "count of people"

    if "income" in col_lower:
        return "yearly income (EUR)"

    #Synthetic fields Z_CostContact / Z_Revenue (from marketing cost model)
    if col_lower.startswith("z_"):
        return "synthetic score (unit unspecified)"

    if col_lower == "id":
        return "identifier (no unit)"

    if "year_birth" in col_lower:
        return "year"

    # fallback: numeric unitless
    if np.issubdtype(series.dropna().dtype, np.number):
        return "unitless numeric value"

    # fallback for categorical
    return "categorical label (no unit)"


# apply the function to every column
units = {col: infer_unit(col, ifood_df[col]) for col in ifood_df.columns}

units_df = pd.DataFrame(list(units.items()), columns=["Attribute", "Unit"])
print(units_df)

#############################################
# Documentation: Identify Attribute Units Activity
#############################################

start_time_iu = now()
# Unit inference already performed above
end_time_iu = now()

identify_units_code_writer = student_a
iu_ass_uuid_executor = "c7d9e3f4-6a5b-4c8d-9e0f-2a3b4c5d6e7f"  # Generate once
iu_ass_uuid_writer = "d8e0f4a5-7b6c-5d9e-0f1a-3b4c5d6e7f8a"  # Generate once

iu_comment = """
Analysis of attribute units in the iFood marketing dataset. Each attribute was assigned a unit based on its semantic meaning:
- Identifier: ID column with no unit
- Year: Birth year
- Calendar date: Customer enrollment date
- Days: Time since last purchase (recency)
- Monetary amount (EUR): All spending columns (wines, fruits, meat, fish, sweets, gold)
- Count: Number of purchases, deals, web visits
- Count of people: Number of kids/teens at home
- Yearly income (EUR): Customer income
- Binary indicator (0/1): Campaign acceptance flags, complain, response
- Synthetic score: Z_CostContact and Z_Revenue (marketing model scores)
- Categorical label: Education and Marital_Status
"""

identify_units_activity = [
    f':identify_attribute_units rdf:type prov:Activity .',
    f':identify_attribute_units sc:isPartOf :data_understanding_phase .',
    f':identify_attribute_units rdfs:label "Identify Attribute Units" .',
    f':identify_attribute_units rdfs:comment """{iu_comment}""" .',
    f':identify_attribute_units prov:startedAtTime "{start_time_iu}"^^xsd:dateTime .',
    f':identify_attribute_units prov:endedAtTime "{end_time_iu}"^^xsd:dateTime .',
    
    # Executor association
    f':identify_attribute_units prov:qualifiedAssociation :{iu_ass_uuid_executor} .',
    f':{iu_ass_uuid_executor} prov:agent :{executed_by} .',
    f':{iu_ass_uuid_executor} rdf:type prov:Association .',
    f':{iu_ass_uuid_executor} prov:hadRole :{code_executor_role} .',
    
    # Writer association
    f':identify_attribute_units prov:qualifiedAssociation :{iu_ass_uuid_writer} .',
    f':{iu_ass_uuid_writer} prov:agent :{identify_units_code_writer} .',
    f':{iu_ass_uuid_writer} rdf:type prov:Association .',
    f':{iu_ass_uuid_writer} prov:hadRole :{code_writer_role} .',
    
    # Input: raw data
    f':identify_attribute_units prov:used :raw_data .',
    
    # Output: attribute units report entity
    f':attribute_units_report rdf:type prov:Entity .',
    f':attribute_units_report prov:wasGeneratedBy :identify_attribute_units .',
    f':attribute_units_report rdfs:label "Attribute Units Classification Report" .',
    f':attribute_units_report rdfs:comment "Classification of 29 attributes with their respective units and measurement semantics." .',
]
engine.insert(identify_units_activity, prefixes=prefixes)

print("Attribute units documentation inserted successfully.")


              Attribute                                Unit
0                    ID                identifier (no unit)
1            Year_Birth                                year
2             Education         categorical label (no unit)
3        Marital_Status         categorical label (no unit)
4                Income                 yearly income (EUR)
5               Kidhome                     count of people
6              Teenhome                     count of people
7           Dt_Customer          calendar date (YYYY-MM-DD)
8               Recency                                days
9              MntWines               monetary amount (EUR)
10            MntFruits               monetary amount (EUR)
11      MntMeatProducts               monetary amount (EUR)
12      MntFishProducts               monetary amount (EUR)
13     MntSweetProducts               monetary amount (EUR)
14         MntGoldProds               monetary amount (EUR)
15    NumDealsPurchases                 

In [16]:
# Dictionary of attribute semantics for ml_project1_data.csv
attribute_semantics = {
    "ID": "Unique customer identifier.",
    "Year_Birth": "Customer's year of birth.",
    "Education": "Customer's highest education level.",
    "Marital_Status": "Customer's marital status.",
    "Income": "Annual household income of the customer.",
    "Kidhome": "Number of children living in the customer's household.",
    "Teenhome": "Number of teenagers living in the customer's household.",
    "Dt_Customer": "Date when the customer first enrolled with the company.",
    "Recency": "Number of days since the customer's last purchase.",
    "MntWines": "Amount spent on wine products in the last 2 years.",
    "MntFruits": "Amount spent on fruit products in the last 2 years.",
    "MntMeatProducts": "Amount spent on meat products in the last 2 years.",
    "MntFishProducts": "Amount spent on fish products in the last 2 years.",
    "MntSweetProducts": "Amount spent on sweet products in the last 2 years.",
    "MntGoldProds": "Amount spent on luxury ('gold') products in the last 2 years.",
    "NumDealsPurchases": "Number of purchases made using discounts.",
    "NumWebPurchases": "Number of purchases made through the company website.",
    "NumCatalogPurchases": "Number of purchases made via catalog orders.",
    "NumStorePurchases": "Number of purchases made in physical stores.",
    "NumWebVisitsMonth": "Number of website visits in the last month.",
    "AcceptedCmp3": "Whether the customer accepted the 3rd campaign (0/1).",
    "AcceptedCmp4": "Whether the customer accepted the 4th campaign (0/1).",
    "AcceptedCmp5": "Whether the customer accepted the 5th campaign (0/1).",
    "AcceptedCmp1": "Whether the customer accepted the 1st campaign (0/1).",
    "AcceptedCmp2": "Whether the customer accepted the 2nd campaign (0/1).",
    "Complain": "Whether the customer had a complaint in the last 2 years.",
    "Z_CostContact": "Administrative constant related to contact cost (no analytical meaning).",
    "Z_Revenue": "Administrative constant related to revenue baseline (no analytical meaning).",
    "Response": "Whether the customer accepted the most recent marketing campaign.",
}
attribute_semantics

#############################################
# Documentation: Attribute Semantics Activity
#############################################

start_time_as = now()
# Attribute semantics already defined above
end_time_as = now()

attribute_semantics_code_writer = student_a
as_ass_uuid_executor = "f1a2b3c4-5d6e-7f8a-9b0c-1d2e3f4a5b6c"  # Generate once
as_ass_uuid_writer = "a2b3c4d5-6e7f-8a9b-0c1d-2e3f4a5b6c7d"  # Generate once

as_comment = f"""
Documentation of attribute semantics for the iFood marketing dataset:
{chr(10).join(f'- {k}: {v}' for k, v in attribute_semantics.items())}
"""

# Document the attribute semantics activity
attribute_semantics_activity = [
    f':identify_attribute_semantics rdf:type prov:Activity .',
    f':identify_attribute_semantics sc:isPartOf :data_understanding_phase .',
    f':identify_attribute_semantics rdfs:label "Identify Attribute Semantics" .',
    f':identify_attribute_semantics rdfs:comment """{as_comment}""" .',
    f':identify_attribute_semantics prov:startedAtTime "{start_time_as}"^^xsd:dateTime .',
    f':identify_attribute_semantics prov:endedAtTime "{end_time_as}"^^xsd:dateTime .',
    
    # Executor association
    f':identify_attribute_semantics prov:qualifiedAssociation :{as_ass_uuid_executor} .',
    f':{as_ass_uuid_executor} prov:agent :{executed_by} .',
    f':{as_ass_uuid_executor} rdf:type prov:Association .',
    f':{as_ass_uuid_executor} prov:hadRole :{code_executor_role} .',
    
    # Writer association
    f':identify_attribute_semantics prov:qualifiedAssociation :{as_ass_uuid_writer} .',
    f':{as_ass_uuid_writer} prov:agent :{attribute_semantics_code_writer} .',
    f':{as_ass_uuid_writer} rdf:type prov:Association .',
    f':{as_ass_uuid_writer} prov:hadRole :{code_writer_role} .',
    
    # Input: raw data
    f':identify_attribute_semantics prov:used :raw_data .',
    
    # Output: attribute semantics report entity
    f':attribute_semantics_report rdf:type prov:Entity .',
    f':attribute_semantics_report prov:wasGeneratedBy :identify_attribute_semantics .',
    f':attribute_semantics_report rdfs:label "Attribute Semantics Report" .',
    f':attribute_semantics_report rdfs:comment "Business context and semantic descriptions for all 29 attributes in the dataset." .',
]
engine.insert(attribute_semantics_activity, prefixes=prefixes)

# Create a combined semantics dataframe for display
semantics_df = pd.DataFrame(list(attribute_semantics.items()), columns=["Attribute", "Semantic_Description"])
display(semantics_df)

print("Attribute semantics documentation inserted successfully.")


,Attribute,Semantic_Description
0,ID,Unique customer identifier.
1,Year_Birth,Customer's year of birth.
2,Education,Customer's highest education level.
3,Marital_Status,Customer's marital status.
4,Income,Annual household income of the customer.
5,Kidhome,Number of children living in the customer's ho...
6,Teenhome,Number of teenagers living in the customer's h...
7,Dt_Customer,Date when the customer first enrolled with the...
8,Recency,Number of days since the customer's last purch...
9,MntWines,Amount spent on wine products in the last 2 ye...


Attribute semantics documentation inserted successfully.


In [17]:
check_outliers_code_writer = student_a

def check_outliers(data: pd.DataFrame, threshold=3.0, columns=('number',)) -> dict:
    results = {}

    ### DIRTY HACK
    ### REPLACE WITH YOUR ACTUAL OUTLIER CHECKING

    tmp = data.copy()
    tmp = tmp.reset_index(drop=True)

    for col in columns:
        values = tmp[col].astype(float)

        mean = values.mean()
        std = values.std()

        if std == 0 or np.isnan(std):
            results[col] = []
            continue

        z_scores = (values - mean) / std

        mask = np.abs(z_scores) > threshold
        outliers = values[mask].index

        outlier_info = [
            {
                'index': int(idx),
                'z_score': float(z_scores.loc[idx])
            }
            for idx in outliers
        ]

        results[col] = outlier_info

    return results

start_time_co = now()
outliers_report = check_outliers(data, threshold=2.2)
end_time_co = now()

start_time_ho = now()
print(outliers_report)
end_time_ho = now()

#############################################
# Documentation
#############################################
# There are three steps involved in this process:
# 1. activity creates a figure, report etc. => in this case a report
# 2. activity inspects the outcome and derives decisions => in this case to remove the outliers that were found
# 3. activity follows up on the decision by changing the data => will be done in the data preparation phase

# 1. Activty: Checking for outliers and creating the report
co_ass_uuid_executor = "15085e9d-15f1-4727-9b6e-776dd07fcd08"
check_outliers_executor = [
    f':check_outliers prov:qualifiedAssociation :{co_ass_uuid_executor} .',
    f':{co_ass_uuid_executor} prov:agent :{executed_by} .',
    f':{co_ass_uuid_executor} rdf:type prov:Association .',
    f':{co_ass_uuid_executor} prov:hadRole :{code_executor_role} .',
]
engine.insert(check_outliers_executor, prefixes=prefixes)

co_ass_uuid_writer = "cd4970df-9f40-4bb1-8fad-e4dc4fcdd284"
co_comment = """
Identifying outliers with a dirty hack that uses the z-score of each row within in column and reports all values 
with a z-score higher than 2.2 as an outlier, which is not a reasonable threshold but used here to avoid not 
finding any outliers for demonstration purposes.
"""
check_outliers_activity = [
    ':check_outliers rdf:type prov:Activity .',
    ':check_outliers sc:isPartOf :data_understanding_phase .',
    ':check_outliers rdfs:comment \'Data Understanding\' .',
    f':check_outliers rdfs:comment """{co_comment}""" .', 
    f':check_outliers prov:startedAtTime "{start_time_co}"^^xsd:dateTime .',
    f':check_outliers prov:endedAtTime "{end_time_co}"^^xsd:dateTime .',
    f':check_outliers prov:qualifiedAssociation :{co_ass_uuid_writer} .',
    f':{co_ass_uuid_writer} prov:agent :{check_outliers_code_writer} .',
    f':{co_ass_uuid_writer} rdf:type prov:Association .',
    f':{co_ass_uuid_writer} prov:hadRole :{code_writer_role} .',
    ':check_outliers prov:used :data .',
    ':outlier_report rdf:type prov:Entity .',
    f':outlier_report rdfs:comment """{json.dumps(outliers_report, indent=2)}""" .',
    ':outlier_report prov:wasGeneratedBy :check_outliers .',
    # ...
]
engine.insert(check_outliers_activity, prefixes=prefixes)

# 2. Activity: Inspecting the report and taking a decision on what to do
ior_ass_uuid_executor = "6eaa2c0a-e592-4d85-b37f-d695844910cf"
ior_comment = """
After inspecting the report the decision has been made to remove all outliers that were identfied for demonstration purpose3
"""
inspect_outlier_report_executor = student_a
inspect_outlier_report_activity = [
    ':inspect_outlier_report rdf:type prov:Activity .',
    ':inspect_outlier_report rdfs:comment \'Data Understanding\' .',
    f':inspect_outlier_report rdfs:comment """{co_comment}""" .', 
    f':inspect_outlier_report prov:startedAtTime "{start_time_co}"^^xsd:dateTime .',
    f':inspect_outlier_report prov:endedAtTime "{end_time_co}"^^xsd:dateTime .',
    f':inspect_outlier_report prov:qualifiedAssociation :{ior_ass_uuid_executor} .',
    f':{ior_ass_uuid_executor} prov:agent :{inspect_outlier_report_executor} .',
    f':{ior_ass_uuid_executor} rdf:type prov:Association .',
    f':{ior_ass_uuid_executor} prov:hadRole :{code_executor_role} .',
    ':inspect_outlier_report prov:used :outlier_report .',
    ':outlier_decision rdf:type prov:Entity .',
    f':outlier_decision rdfs:comment """Removing all outliers for demonstration purposes.""" .',
    ':outlier_decision prov:wasGeneratedBy :inspect_outlier_report .',
    # ...
]
engine.insert(inspect_outlier_report_activity, prefixes=prefixes)

NameError: name 'data' is not defined

**Continue with other tasks of the Data Understanding phase such as checking the distribution, skewness, plausibility of values, etc...**

## Data Preparation

In [ ]:
## Each Activity that follows is part of the Data Preparation Phase

data_preparation_phase_executor = [
f':data_preparation_phase rdf:type prov:Activity .',
f':data_preparation_phase rdfs:label "Data Preparation Phase" .', 
]
engine.insert(data_preparation_phase_executor, prefixes=prefixes)

In [ ]:
handle_outliers_code_writer = student_b
def handle_outliers(df:pd.DataFrame, outliers_report: dict) -> pd.DataFrame:
    # REMOVE OUTLIERS
    return df

start_time_td = now()
handle_outliers(data, outliers_report)
end_time_td = now()

#############################################
# Documentation
#############################################
# This is the continuation of the example from the Data Understanding phase above.
# There are three steps involved in this process:
# 1. activity creates a figure, report etc. => already done in data understanding phase
# 2. activity inspects the outcome and derives decisions => already done in data understanding phase
# 3. activity follows up on the decision by changing the data => in this case by removing the the outliers that were found

ro_ass_uuid_executor = "ec7e81e1-86ea-475a-a8d4-c7d8ee535488"
handle_outliers_executor = [
    f':handle_outliers prov:qualifiedAssociation :{ro_ass_uuid_executor} .',
    f':{ro_ass_uuid_executor} prov:agent :{executed_by} .',
    f':{ro_ass_uuid_executor} rdf:type prov:Association .',
    f':{ro_ass_uuid_executor} prov:hadRole :{code_executor_role} .',
]
engine.insert(handle_outliers_executor, prefixes=prefixes)

td_ass_uuid_writer = "1405f15a-3545-4014-a962-637f3c10a137"
td_comment = """
Removing all outliers that were identifying in the Data Understanding Phase.
"""
handle_outliers_activity = [
    ':handle_outliers rdf:type prov:Activity .',
    ':handle_outliers sc:isPartOf :data_preparation_phase .',
    ':handle_outliers rdfs:comment \'Data Preparation\' .', 
    f':handle_outliers rdfs:comment """{td_comment}""" .', 
    f':handle_outliers prov:startedAtTime "{start_time_td}"^^xsd:dateTime .',
    f':handle_outliers prov:endedAtTime "{end_time_td}"^^xsd:dateTime .',
    f':handle_outliers prov:qualifiedAssociation :{td_ass_uuid_writer} .',
    f':{td_ass_uuid_writer} prov:agent :{handle_outliers_code_writer} .',
    f':{td_ass_uuid_writer} rdf:type prov:Association .',
    f':{td_ass_uuid_writer} prov:hadRole :{code_writer_role} .',
    ':handle_outliers prov:used :data .',
    ':handle_outliers prov:used :outlier_decision .',
    ':cleaned_data rdf:type prov:Entity .',
    ':cleaned_data prov:wasGeneratedBy :handle_outliers .',
    ':cleaned_data prov:wasDerivedFrom :data .',
]
engine.insert(handle_outliers_activity, prefixes=prefixes)

**Continue with other tasks of the Data Preparation phase such as binning, scaling etc...**

In [ ]:
# Your final transformed dataset should also be documented appropriately using Croissant, SI, etc.

prepared_data_triples = [
    ':prepared_data rdf:type prov:Entity .',
    ':prepared_data prov:wasDerivedFrom :cleaned_data .',
    ':prepared_data rdf:type sc:Dataset .',
    # ....
]
engine.insert(prepared_data_triples, prefixes=prefixes)

## Modeling

In [ ]:
## Each Activity that follows is part of the Modeling Phase

modeling_phase_executor = [
f':modeling_phase rdf:type prov:Activity .',
f':modeling rdfs:label "Modeling Phase" .', 
]
engine.insert(modeling_phase_executor, prefixes=prefixes)


In [ ]:
model_data_code_writer = student_a

#############################################
# Documentation 4a
#############################################

dma_ass_uuid_writer = "b3e840ab-ac23-415e-bd9c-6d00bb79c37a"
dma_comment = """
...
"""

identify_data_mining_algorithm_activity = [
    f':define_algorithm rdf:type prov:Activity .',
    f':define_algorithm sc:isPartOf :modeling_phase .',
    f':define_algorithm rdfs:comment """{dma_comment}""" .',
    f':define_algorithm prov:qualifiedAssociation :{dma_ass_uuid_writer} .',
    f':{dma_ass_uuid_writer} prov:agent :{model_data_code_writer} .',
    f':{dma_ass_uuid_writer} rdf:type prov:Association .',
    f':{dma_ass_uuid_writer} prov:hadRole :{code_writer_role} .',
    
    # example algorithm definition
    f':random_forest_algorithm rdf:type mls:Algorithm .',
    f':random_forest_algorithm rdfs:label "Random Forest Algorithm" .',

    # example implementation
    f':random_forrest_classifier_implementation rdf:type mls:Implementation .',
    f':random_forrest_classifier_implementation rdfs:label "Scikit-learn RandomForestClassifier" .',
    f':random_forrest_classifier_implementation mls:implements :random_forest_algorithm .',
    f':random_forrest_classifier_implementation prov:wasGeneratedBy :define_algorithm .',

    
    # you can also define your Evaluation Measures here
    
    # example evaluation 
    f':r2_score_measure rdf:type mls:EvaluationMeasure .',
    f':r2_score_measure rdfs:label "R-squared Score" .',
    f':r2_score_measure rdfs:comment "xxx" .',
    f':r2_score_measure prov:wasGeneratedBy :define_algorithm .',

    
]
engine.insert(identify_data_mining_algorithm_activity, prefixes=prefixes)

In [ ]:
#############################################
# Documentation 4b
#############################################

hp_ass_uuid_writer = "fff582a8-c5cd-4030-978b-9f56b603167c"
hp_comment = """
...
"""
identify_hp_activity = [
    f':identify_hyperparameters rdf:type prov:Activity .',
    f':identify_hyperparameters sc:isPartOf :modeling_phase .',
    f':identify_hyperparameters rdfs:comment """{hp_comment}""" .',
    f':identify_hyperparameters prov:qualifiedAssociation :{hp_ass_uuid_writer} .',
    f':{hp_ass_uuid_writer} prov:agent :{model_data_code_writer} .',
    f':{hp_ass_uuid_writer} rdf:type prov:Association .',
    f':{hp_ass_uuid_writer} prov:hadRole :{code_writer_role} .',
    
    # example parameter
    f':hp_learning_rate rdf:type mls:HyperParameter .',
    f':hp_learning_rate rdfs:label "Learning Rate" .',
    f':hp_learning_rate rdfs:comment "..." .',
    f':random_forrest_classifier_implementation mls:hasHyperParameter :hp_learning_rate .',
    f':hp_learning_rate prov:wasGeneratedBy :identify_hyperparameters .',

    # continue with your identified hyperparameters
    
]
engine.insert(identify_hp_activity, prefixes=prefixes)

In [ ]:
def split_data(df: pd.DataFrame):
    #do something
    return 'train_set', 'validation_set', 'test_set'

#############################################
# Documentation 4c
#############################################

### Define Train/Validation/Test splits
split_ass_uuid_writer = "fb58ae6c-9d58-44c9-ac7e-529111bdf7fc"
split_comment = """
...
"""
## Use your prepared dataset
input_dataset = ":prepared_data" 

define_split_activity = [
    f':define_data_split rdf:type prov:Activity .',
    f':define_data_split sc:isPartOf :modeling_phase .',
    f':define_data_split rdfs:comment "Train/Validation/Test Split Definition" .',
    f':define_data_split rdfs:comment """{split_comment}""" .',
    f':define_data_split prov:qualifiedAssociation :{split_ass_uuid_writer} .',
    f':{split_ass_uuid_writer} prov:agent :{model_data_code_writer} .',
    f':{split_ass_uuid_writer} rdf:type prov:Association .',
    f':{split_ass_uuid_writer} prov:hadRole :{code_writer_role} .',
    f':define_data_split prov:used {input_dataset} .',
    
    # Training Set
    f':training_set rdf:type sc:Dataset .',
    f':training_set rdfs:label "Training Set" .',
    f':training_set prov:wasGeneratedBy :define_data_split .',
    f':training_set prov:wasDerivedFrom {input_dataset} .',
    f':training_set rdfs:comment "Contains xx samples" .', 

    # Validation Set
    f':validation_set rdf:type sc:Dataset .',
    f':validation_set rdfs:label "Validation Set" .',
    f':validation_set prov:wasGeneratedBy :define_data_split .',
    f':validation_set prov:wasDerivedFrom {input_dataset} .',
    f':validation_set rdfs:comment "Contains xx samples" .', 

    # Test Set
    f':test_set rdf:type sc:Dataset .',
    f':test_set rdfs:label "Test Set" .',
    f':test_set prov:wasGeneratedBy :define_data_split .',
    f':test_set prov:wasDerivedFrom {input_dataset} .',
    f':test_set rdfs:comment "Contains xx samples" .', 

    
]
engine.insert(define_split_activity, prefixes=prefixes)

In [ ]:
def train_and_finetune_model(training_set, validation_set):
    # do something here

    # Try to automate as much documentation work as possible.
    # Define your training runs with their respective hyperparameter settings, etc.
    # Document each time a training run, model, its hp_settings, evaluations, ...  
    # Create performance figures/graphs

    return 'Find most suitable model'


start_time_tafm = now()
# train_and_finetune_model()
end_time_tafm = now() 


#############################################
# Documentation 4d & e & f
#############################################

tafm_ass_uuid_writer = "21d60fe3-c9ab-4a0a-bae7-b9fe9653c755"
tafm_comment = """
...
"""

# EXAMPLE output from your training
training_run1 = "run_1" 
model_run1 = "model_run1"
hp1_setting_run1 = "hp_setting_run1"
eval_train_run1 = "metric_train_run1"
eval_validation_run1 = "metric_validation_run1"


train_model_activity = [
    # Activity 
    f':train_and_finetune_model rdf:type prov:Activity .',
    f':train_and_finetune_model sc:isPartOf :modeling_phase .',
    f':train_and_finetune_model rdfs:comment """{tafm_comment}""" .',
    f':train_and_finetune_model prov:startedAtTime "{start_time_tafm}"^^xsd:dateTime .',
    f':train_and_finetune_model prov:endedAtTime "{end_time_tafm}"^^xsd:dateTime .',
    f':train_and_finetune_model prov:qualifiedAssociation :{tafm_ass_uuid_writer} .',
    f':{tafm_ass_uuid_writer} prov:agent :{model_data_code_writer} .',
    f':{tafm_ass_uuid_writer} rdf:type prov:Association .',
    f':{tafm_ass_uuid_writer} prov:hadRole :{code_writer_role} .',
    
    ########################################
    # ONE model run - automate everything below!

    # Parameter settings
    f':{hp1_setting_run1} rdf:type mls:HyperParameterSetting .',
    f':{hp1_setting_run1} mls:specifiedBy :hp_learning_rate .',
    f':{hp1_setting_run1} mls:hasValue "1.23"^^xsd:double .',
    f':{hp1_setting_run1} prov:wasGeneratedBy :train_and_finetune_model .',
    # add your further parameters

    # Describe your Run
    f':{training_run1} rdf:type mls:Run .',
    f':{training_run1} sc:isPartOf :train_and_finetune_model .',
    f':{training_run1} mls:realizes :random_forest_algorithm .',
    f':{training_run1} rdf:label "Training Run 1 with..." .',
    f':{training_run1} mls:executes :your_implementation .', 
    f':{training_run1} mls:hasInput :training_set .',
    f':{training_run1} mls:hasInput :validation_set .',
    f':{training_run1} mls:hasInput :{hp1_setting_run1} .',     
    # list all your used parameters here
    f':{training_run1} mls:hasOutput :{model_run1} .',
    f':{training_run1} mls:hasOutput :{eval_train_run1} .',
    f':{training_run1} mls:hasOutput :{eval_validation_run1} .',

    # Describe your Model
    f':{model_run1} rdf:type mls:Model .',
    f':{model_run1} prov:label "xxx" .',
    f':{model_run1} prov:wasGeneratedBy :{training_run1} .',
    f':{model_run1} mlso:trainedOn :training_set .',
    f':{model_run1} mlso:hasAlgorithmType :random_forest_algorithm .',

    # Describe your evaluations
    # You can have multiple evaluations per model 
    f':{eval_train_run1} rdf:type mls:ModelEvaluation .',
    f':{eval_train_run1} prov:wasGeneratedBy :{training_run1} .',
    f':{eval_train_run1} mls:hasValue "1.23"^^xsd:double .',
    f':{eval_train_run1} mls:specifiedBy :r2_score_measure .',
    f':{eval_train_run1} prov:used :training_set .',

    f':{eval_validation_run1} rdf:type mls:ModelEvaluation .',
    f':{eval_validation_run1} prov:wasGeneratedBy :{training_run1} .',
    f':{eval_validation_run1} mls:hasValue "1.23"^^xsd:double .',
    f':{eval_validation_run1} mls:specifiedBy :r2_score_measure .',
    f':{eval_validation_run1} prov:used :validation_set .',

    # Dont forget to document any visualizations

]
engine.insert(train_model_activity, prefixes=prefixes)


In [ ]:
def retrain_model_full_data(training_set, validation_set):
    
    # create your
    return "Final Trained Model"


start_time_tafm = now()
# train_and_finetune_model()
end_time_tafm = now() 


#############################################
# Documentation 4g
#############################################

retrain_ass_uuid_writer = "96815ee0-524c-437b-b5fa-2e15b945c993" # Generate once

final_training_activity = ":retrain_final_model"
final_model = ":final_model_entity"

# Document the retraining activity.
# Hint: This activity is still part of the :modeling_phase

retrain_documentation = [
    # your documentation here    
]
engine.insert(retrain_documentation, prefixes=prefixes)


## Evaluation

In [ ]:
## Each Activity that follows is part of the Evaluation Phase

evaluation_phase_executor = [
f':evaluation_phase rdf:type prov:Activity .',
f':evaluation_phase rdfs:label "Evaluation Phase" .', 
]
engine.insert(evaluation_phase_executor, prefixes=prefixes)

In [ ]:
eval_code_writer = student_b
def evaluate_on_test_data(final_model, test_set):

    # Predict and evaluation on test data
        
    return 'Performance'

start_time_eval = now()
#evaluate_on_test_data()
end_time_eval = now() 

#############################################
# Documentation
#############################################

eval_ass_uuid = "7f1431e9-feed-429a-92ed-c131b23cbe79" # Generate once
final_model = ":final_model_entity" 
test_set = ":test_set" 

eval_comment = """
...
"""

evaluate_activity = [
    f':evaluate_final_model rdf:type prov:Activity .',
    f':evaluate_final_model sc:isPartOf :evaluation_phase .',
    f':evaluate_final_model rdfs:label "Final Model Evaluation on Test Set" .',
    f':evaluate_final_model rdfs:comment """{eval_comment}""" .',
    f':evaluate_final_model prov:startedAtTime "{start_time_eval}"^^xsd:dateTime .',
    f':evaluate_final_model prov:endedAtTime "{end_time_eval}"^^xsd:dateTime .',
    f':evaluate_final_model prov:qualifiedAssociation :{eval_ass_uuid} .',
    
    f':{eval_ass_uuid} prov:agent :{eval_code_writer} .',
    f':{eval_ass_uuid} rdf:type prov:Association .',
    f':{eval_ass_uuid} prov:hadRole :{code_writer_role} .',

    # Inputs
    f':evaluate_final_model prov:used {final_model} .',
    f':evaluate_final_model prov:used {test_set} .',
    
    # Reference to Data Mining Success Criteria from Phase 1
    f':evaluate_final_model prov:used :bu_data_mining_success_criteria .',

    # Document you final model performance
 
    # Hint: you evaluate bias in this way:
    f':bias_evaluation_result rdf:type mls:ModelEvaluation .',
    f':bias_evaluation_result prov:wasGeneratedBy :evaluate_final_model .',
    f':bias_evaluation_result rdfs:label "Bias Analysis" .',
    f':bias_evaluation_result rdfs:comment "..." .',
    
]
engine.insert(evaluate_activity, prefixes=prefixes)

## Deployment

In [ ]:
## Each Activity that follows is part of the Deployment Phase

deployment_phase_executor = [
f':deployment_phase rdf:type prov:Activity .',
f':deployment_phase rdfs:label "Deployment Phase" .', 
]
engine.insert(deployment_phase_executor, prefixes=prefixes)

In [ ]:
#############################################
# Documentation
#############################################

comparison_and_recommendations_comment = """
...
"""

ethical_aspects_comment = """
...
"""

monitoring_plan_comment = """
...
"""

reproducibility_reflection_comment = """
...
"""

dep_ass_uuid_executor = "72a921e0-1234-4567-89ab-cdef01234567" # Generate once
deployment_executor = [
f':plan_deployment rdf:type prov:Activity .',
f':plan_deployment sc:isPartOf :deployment_phase .', # Connect to Parent Phase
f':plan_deployment rdfs:label "Plan Deployment"@en .',

f':plan_deployment prov:qualifiedAssociation :{dep_ass_uuid_executor} .',
f':{dep_ass_uuid_executor} prov:agent :{executed_by} .',
f':{dep_ass_uuid_executor} rdf:type prov:Association .',
f':{dep_ass_uuid_executor} prov:hadRole :{code_executor_role} .', 
]
engine.insert(deployment_executor, prefixes=prefixes)


deployment_data_executor = [
#6a
f':dep_recommendations rdf:type prov:Entity .',
f':dep_recommendations prov:wasGeneratedBy :plan_deployment .',
f':dep_recommendations rdfs:label "6a Business Objectives Reflection and Deployment Recommendations" .',
f':dep_recommendations rdfs:comment """{comparison_and_recommendations_comment}""" .',
#6b
f':dep_ethical_risks rdf:type prov:Entity .',
f':dep_ethical_risks prov:wasGeneratedBy :plan_deployment .',
f':dep_ethical_risks rdfs:label "6b Ethical Aspects and Risks" .',
f':dep_ethical_risks rdfs:comment """{ethical_aspects_comment}""" .',
#6c
f':dep_monitoring_plan rdf:type prov:Entity .',
f':dep_monitoring_plan prov:wasGeneratedBy :plan_deployment .',
f':dep_monitoring_plan rdfs:label "6c Monitoring Plan" .',
f':dep_monitoring_plan rdfs:comment """{monitoring_plan_comment}""" .',
#6d
f':dep_reproducibility_reflection rdf:type prov:Entity .',
f':dep_reproducibility_reflection prov:wasGeneratedBy :plan_deployment .',
f':dep_reproducibility_reflection rdfs:label "6d Reproducibility Reflection" .',
f':dep_reproducibility_reflection rdfs:comment """{reproducibility_reflection_comment}""" .',

]
engine.insert(deployment_data_executor, prefixes=prefixes)

# Generate Latex Report

The following cells give you an example of how to automatically create a Latex Report from your provenance documentation.

Feel free to use the example provided. If you use it, you should adapt and extend it with relevant sections/tables/plots/... 

In [18]:
base_iri = f"https://starvers.ec.tuwien.ac.at/BI2025/{group_id}/"

In [19]:
# This cell includes cleaning functions

from datetime import datetime

def latex_escape(text: str | None) -> str:
    if text is None: return ""
    text = str(text)
    text = text.replace("\\", r"\textbackslash{}")
    pairs = [
        ("&", r"\&"), ("%", r"\%"), ("$", r"\$"), ("#", r"\#"), 
        ("_", r"\_"), ("{", r"\{"), ("}", r"\}"), 
        ("~", r"\textasciitilde{}"), ("^", r"\textasciicircum{}")
    ]
    for k, v in pairs:
        text = text.replace(k, v)
    return text

def clean_rdf(x) -> str:
    if hasattr(x, "toPython"): return str(x.toPython())
    if x is None: return ""
    s = str(x).strip()
    s = s.strip('"').strip("'")
    s = s.strip()
    if "^^" in s:
        s = s.split("^^")[0].strip('"')
        
    return s

def fmt_iso(ts: str) -> str:
    if not ts: return ""
    try:
        clean_ts = ts.split("^^")[0].strip('"')
        clean_ts = clean_ts.replace("Z", "+00:00") if clean_ts.endswith("Z") else clean_ts
        return datetime.fromisoformat(clean_ts).strftime("%Y-%m-%d %H:%M:%S")
    except:
        return latex_escape(str(ts))

In [20]:
# This cell includes exemplary queries for different phases
time.tzname = ('Europe/Vienna', 'Europe/Vienna')

### Author Block
author_query = f"""
{prefix_header}
PREFIX iao: <http://purl.obolibrary.org/obo/>

SELECT DISTINCT ?uri ?given ?family ?matr WHERE {{
  VALUES ?uri {{ :{student_a} :{student_b} }}
  
  ?uri a foaf:Person .
  ?uri foaf:givenName ?given .
  ?uri foaf:familyName ?family .
  ?uri iao:IAO_0000219 ?matr .
}}
"""

res_authors = engine.query(author_query)
author_block_latex = ""

if not res_authors.empty: # type:ignore
    for _, row in res_authors.iterrows(): # type:ignore

        uri_str = str(row['uri'])
        given = latex_escape(clean_rdf(row['given']))
        family = latex_escape(clean_rdf(row['family']))
        matr = latex_escape(clean_rdf(row['matr']))
        if student_a in uri_str:
            responsibility = "Student A"
        elif student_b in uri_str:
            responsibility = "Student B"
        else:
            responsibility = "Student"
        
        author_block_latex += rf"""
          \author{{{given} {family}}}
          \authornote{{{responsibility}, Matr.Nr.: {matr}}}
          \affiliation{{
            \institution{{TU Wien}}
            \country{{Austria}}
          }}
          """

### Business Understanding example
# Query for data source comment
bu_ds_query = f"""
{prefix_header}
SELECT ?ds_comment WHERE {{
  :bu_data_source_and_scenario rdfs:comment ?ds_comment .
}}
"""

# Query for business objectives comment
bu_bo_query = f"""
{prefix_header}
SELECT ?bo_comment WHERE {{
  :bu_business_objectives rdfs:comment ?bo_comment .
}}
"""

res_bu_ds = engine.query(bu_ds_query)
res_bu_bo = engine.query(bu_bo_query)

# Get data source comment
bu_data_source = ""
if not res_bu_ds.empty: # type:ignore
    bu_data_source = latex_escape(clean_rdf(res_bu_ds.iloc[0].get("ds_comment", "")))

# Get business objectives comment  
bu_objectives = ""
if not res_bu_bo.empty: # type:ignore
    bu_objectives = latex_escape(clean_rdf(res_bu_bo.iloc[0].get("bo_comment", "")))

# Debug output to verify data retrieval
print(f"Data Source retrieved: {len(bu_data_source)} characters")
print(f"Business Objectives retrieved: {len(bu_objectives)} characters")


### Data Understanding examples
# Example Dataset Description
du_desc_query = f"""
{prefix_header}
SELECT ?desc WHERE {{ :raw_data sc:description ?desc . }} LIMIT 1
"""
res_du_desc = engine.query(du_desc_query)
row_du_desc = res_du_desc.iloc[0] if not res_du_desc.empty else {} # type:ignore
du_description = latex_escape(clean_rdf(row_du_desc.get("desc", "")))

# Query for Attribute Types Activity
du_attr_types_query = f"""
{prefix_header}
SELECT ?comment WHERE {{
  :identify_attribute_types rdfs:comment ?comment .
}}
"""
res_attr_types = engine.query(du_attr_types_query)
du_attr_types_comment = ""
if not res_attr_types.empty: # type:ignore
    du_attr_types_comment = latex_escape(clean_rdf(res_attr_types.iloc[0].get("comment", "")))

# Query for Attribute Types Statistics
du_attr_stats_query = f"""
{prefix_header}
SELECT ?comment WHERE {{
  :attribute_types_statistics rdfs:comment ?comment .
}}
"""
res_attr_stats = engine.query(du_attr_stats_query)
du_attr_stats_comment = ""
if not res_attr_stats.empty: # type:ignore
    du_attr_stats_comment = latex_escape(clean_rdf(res_attr_stats.iloc[0].get("comment", "")))

# Query for Attribute Units Activity
du_attr_units_query = f"""
{prefix_header}
SELECT ?comment WHERE {{
  :identify_attribute_units rdfs:comment ?comment .
}}
"""
res_attr_units = engine.query(du_attr_units_query)
du_attr_units_comment = ""
if not res_attr_units.empty: # type:ignore
    du_attr_units_comment = latex_escape(clean_rdf(res_attr_units.iloc[0].get("comment", "")))

# Query for Attribute Semantics Activity
du_attr_semantics_query = f"""
{prefix_header}
SELECT ?comment WHERE {{
  :identify_attribute_semantics rdfs:comment ?comment .
}}
"""
res_attr_semantics = engine.query(du_attr_semantics_query)
du_attr_semantics_comment = ""
if not res_attr_semantics.empty: # type:ignore
  raw_comment = clean_rdf(res_attr_semantics.iloc[0].get("comment", ""))
  # Summarize the semantics documentation
  du_attr_semantics_comment = latex_escape(
    "Attribute semantics were documented for all 29 features, including customer demographics (ID, Year_Birth, Education, Marital_Status), "
    "financial data (Income, spending amounts), household composition (Kidhome, Teenhome), "
    "behavioral metrics (Recency, purchase counts, web visits), and campaign response indicators (AcceptedCmp1-5, Response)."
  )

print(f"Attribute Types retrieved: {len(du_attr_types_comment)} characters")
print(f"Attribute Statistics retrieved: {len(du_attr_stats_comment)} characters")
print(f"Attribute Units retrieved: {len(du_attr_units_comment)} characters")
print(f"Attribute Semantics retrieved: {len(du_attr_semantics_comment)} characters")

# Example Feature Columns Table with measurement levels
du_query = f"""
{prefix_header}

SELECT ?name (SAMPLE(?dtypeRaw) as ?dtype) (SAMPLE(?descRaw) as ?desc) (SAMPLE(?measurementRaw) as ?measurement) WHERE {{
  :raw_data cr:recordSet ?rs .
  ?rs cr:field ?field .
  ?field sc:name ?name .
  ?field sc:description ?descRaw .
  ?field cr:dataType ?dtypeRaw .
  OPTIONAL {{ ?field rdfs:comment ?measurementRaw . }}
}} 
GROUP BY ?name
ORDER BY ?name
"""
res_du = engine.query(du_query)
du_rows = []
if not res_du.empty: # type:ignore
    for _, f in res_du.iterrows(): # type:ignore
        dtype_raw = clean_rdf(f.get("dtype", ""))
        if '#' in dtype_raw: dtype = dtype_raw.split('#')[-1]
        elif '/' in dtype_raw: dtype = dtype_raw.split('/')[-1]
        else: dtype = dtype_raw
        
        desc = clean_rdf(f.get("desc", ""))
        row_str = f"{latex_escape(clean_rdf(f['name']))} & {latex_escape(dtype)} & {latex_escape(desc)} \\\\"
        du_rows.append(row_str)
du_table_rows = "\n    ".join(du_rows)

### Modeling example
# Hyperparameters
hp_query = f"""
{prefix_header}

SELECT ?hpName (SAMPLE(?hpValRaw) as ?hpVal) (MAX(?hpDescRaw) as ?hpDesc) WHERE {{
  ?run sc:isPartOf :train_and_finetune_model .
  ?run mls:hasInput ?setting .
  ?setting a mls:HyperParameterSetting .
  ?setting mls:hasValue ?hpValRaw .
  ?setting mls:specifiedBy ?hpDef .
  ?hpDef rdfs:label ?hpName .
  OPTIONAL {{ ?hpDef rdfs:comment ?hpDescRaw . }}
}} 
GROUP BY ?hpName
ORDER BY ?hpName
"""
res_hp = engine.query(hp_query)
hp_rows = []
if not res_hp.empty: #type:ignore
    for _, row in res_hp.iterrows(): #type:ignore
        name = latex_escape(clean_rdf(row['hpName']))
        val  = latex_escape(clean_rdf(row['hpVal']))
        desc = latex_escape(clean_rdf(row.get('hpDesc', '')))
        hp_rows.append(rf"{name} & {desc} & {val} \\")

hp_table_rows = "\n    ".join(hp_rows)

# Run Info
run_query = f"""
{prefix_header}

SELECT ?algoLabel ?start ?end ?metricLabel ?metricVal WHERE {{
  OPTIONAL {{ :train_and_finetune_model prov:startedAtTime ?start ; prov:endedAtTime ?end . }}
  OPTIONAL {{
      ?run sc:isPartOf :train_and_finetune_model .
      ?run mls:realizes ?algo .
      ?algo rdfs:label ?algoLabel .
  }}
  OPTIONAL {{
    ?run sc:isPartOf :train_and_finetune_model .
    ?run mls:hasOutput ?eval .
    ?eval a mls:ModelEvaluation ; mls:hasValue ?metricVal .
    OPTIONAL {{ ?eval mls:specifiedBy ?m . ?m rdfs:label ?metricLabel . }}
  }}
}} LIMIT 1
"""
res_run = engine.query(run_query)
row_run = res_run.iloc[0] if not res_run.empty else {} #type:ignore
mod_algo  = latex_escape(clean_rdf(row_run.get("algoLabel", "")))
mod_start = latex_escape(fmt_iso(clean_rdf(row_run.get("start"))))
mod_end   = latex_escape(fmt_iso(clean_rdf(row_run.get("end"))))
mod_m_lbl = latex_escape(clean_rdf(row_run.get("metricLabel", "")))
raw_val = clean_rdf(row_run.get('metricVal', ''))
mod_m_val = f"{float(raw_val):.4f}" if raw_val else ""

print("Data extraction done.")

Data Source retrieved: 42 characters
Business Objectives retrieved: 3 characters
Attribute Types retrieved: 618 characters
Attribute Statistics retrieved: 324 characters
Attribute Units retrieved: 686 characters
Attribute Semantics retrieved: 341 characters
Attribute Types retrieved: 618 characters
Attribute Statistics retrieved: 324 characters
Attribute Units retrieved: 686 characters
Attribute Semantics retrieved: 341 characters
Data extraction done.
Data extraction done.


The following includes the Latex report itself. It fills in the query-results from the cell before. The ACM Template is already filled. 
Make sure that you update Student A and B accordingly.

In [21]:
latex_content = rf"""\documentclass[sigconf]{{acmart}}

\AtBeginDocument{{ \providecommand\BibTeX{{ Bib\TeX }} }}
\setcopyright{{acmlicensed}}
\copyrightyear{{2025}}
\acmYear{{2025}}
\acmDOI{{XXXXXXX.XXXXXXX}}

\acmConference[BI 2025]{{Business Intelligence}}{{-}}{{-}}

\begin{{document}}

\title{{BI2025 Experiment Report - Group {group_id}}}
%% ---Authors: Dynamically added ---
{author_block_latex}

\begin{{abstract}}
  This report documents the machine learning experiment for Group {group_id}, following the CRISP-DM process model.
\end{{abstract}}

\ccsdesc[500]{{Computing methodologies~Machine learning}}
\keywords{{CRISP-DM, Provenance, Knowledge Graph, Machine Learning}}

\maketitle

%% --- 1. Business Understanding ---
\section{{Business Understanding}}

\subsection{{Data Source and Scenario}}
{latex_escape(data_src_and_scenario_comment)}

\subsection{{Business Objectives}}
{latex_escape(business_objectives_comment)}

\subsection{{Business Success Criteria}}
{latex_escape(business_success_criteria_comment)}

\subsection{{Data Mining Goals}}
{latex_escape(data_mining_goals_comment)}

\subsection{{Data Mining Success Criteria}}
{latex_escape(data_mining_success_criteria_comment)}

%% --- 2. Data Understanding ---
\section{{Data Understanding}}

\subsection{{Attribute Types}}
{du_attr_types_comment}

\subsubsection{{Measurement Level Distribution}}
{du_attr_stats_comment}

\subsection{{Attribute Units}}
{du_attr_units_comment}

\subsection{{Attribute Semantics}}
{du_attr_semantics_comment}

%% --- 3. Data Preparation ---
\section{{Data Preparation}}
\subsection{{Data Cleaning}}
Describe your Data preparation steps here and include respective graph data.


%% --- 4. Modeling ---
\section{{Modeling}}

\subsection{{Hyperparameter Configuration}}
The model was trained using the following hyperparameter settings:

\begin{{table}}[h]
  \caption{{Hyperparameter Settings}}
  \label{{tab:hyperparams}}
  \begin{{tabular}}{{lp{{0.4\linewidth}}l}}
    \toprule
    \textbf{{Parameter}} & \textbf{{Description}} & \textbf{{Value}} \\
    \midrule
    {hp_table_rows}
    \bottomrule
  \end{{tabular}}
\end{{table}}

\subsection{{Training Run}}
A training run was executed with the following characteristics:
\begin{{itemize}}
    \item \textbf{{Algorithm:}} {mod_algo}
    \item \textbf{{Start Time:}} {mod_start}
    \item \textbf{{End Time:}} {mod_end}
    \item \textbf{{Metric:}} {mod_m_lbl} = {mod_m_val}
\end{{itemize}}

%% --- 5. Evaluation ---
\section{{Evaluation}}
% Add evaluation content here

%% --- 6. Deployment ---
\section{{Deployment}}
% Add deployment content here

\section{{Conclusion}}

\end{{document}}
"""

In [22]:
# This cell stores the Latex report to the data/report directory

out_dir = os.path.join("data", "report")
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, "experiment_report.tex")

with open(out_path, "w", encoding="utf-8") as f:
    f.write(latex_content)

print(f"Report written to: {out_path}")

Report written to: data\report\experiment_report.tex
